<a href="https://colab.research.google.com/github/VANvonZHANG/2021-Cloud-Hackathon/blob/main/tutorials-templates/01_Data_Discovery_CMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Data discovery with CMR

In this tutorial you will learn:
- what CMR is;  
- how to use the `requests` package to search data collections and granules;  
- how to parse the results of these searches.


We will focus on datasets in the cloud.  Currently, DAACs with data in the cloud are 'ASF', 'GES_DISC', 'GHRC_DAAC', 'LPCLOUD', 'ORNL_CLOUD', 'POCLOUD'

## What is CMR
CMR is the Common Metadata Repository.  It catalogs all data for NASA's Earth Observing System Data and Information System (EOSDIS).  It is the backend of [Earthdata Search](https://search.earthdata.nasa.gov/search), the GUI search interface you are probably familiar with.  More information about CMR can be found [here](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr).

Unfortunately, the GUI for Earthdata Search is not accessible from a cloud instance - at least not without some work.  Earthdata Search is also not immediately reproducible.  What I mean by that is if you create a search using the GUI you would have to note the search criteria (date range, search area, collection name, etc), take a screenshot, copy the search url, or save the list of data granules returned by the search, in order to recreate the search.  This information would have to be re-entered each time you or someone else wanted to do the search.  You could make typos or other mistakes.  A cleaner, reproducible solution is to search CMR programmatically using the CMR API.

## What is the CMR API
API stands for Application Programming Interface.  It allows applications (software, services, etc) to send information to each other.  A helpful analogy is a waiter in a restaurant.  The waiter takes your drink or food order that you select from the menu, often translated into short-hand, to the bar or kitchen, and then returns (hopefully) with what you ordered when it is ready.

The CMR API accepts search terms such as collection name, keywords, datetime range, and location, queries the CMR database and returns the results.


## How to search CMR from Python
The first step is to import python packages.  We will use:  
- `requests` This package does most of the work for us accessing the CMR API using HTTP methods.
- `pprint` to _pretty print_ the results of the search.  

A more in depth tutorial on `requests` is [here](https://realpython.com/python-requests/)

In [2]:
import requests
from pprint import pprint

Then we need to authenticate with EarthData Login. Since we've already set this up in the previous lesson, here you need to enter your username before executing the cell.

To conduct a search using the CMR API, `requests` needs the url for the root CMR search endpoint.
We'll build this url as a python variable.

In [14]:
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'

CMR allows search by __collections__, which are datasets, and __granules__, which are files that contain data.  Many of the same search parameters can be used for colections and granules but the type of results returned differ.  Search parameters can be found in the [API Documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

Whether we search __collections__ or __granules__ is distinguished by adding `"collections"` or `"granules"` to the url for the root CMR endpoint.

We are going to search collections first, so we add collections to the url.  I'm using a `python` format string here.

In [15]:
url = f'{CMR_OPS}/{"collections"}'

In this first example, I want to retrieve a list of collections that are hosted in the cloud.  Each collection has a `cloud_hosted` parameter that is either True if that collection is in the cloud and False if it is not.  The migration of NASA data to the cloud is a work in progress.  Not all collections tagged as `cloud_hosted` have granules.  To search for only `cloud_hosted` datasets with granules, I also set `has_granules` to `True`.

I also want to get the content in `json` (pronounced "jason") format, so I pass a dictionary to the header keyword argument to say that I want results returned as `json`.

The `.get()` method is used to send this information to the CMR API.  `get()` calls the HTTP method __GET__.

In [16]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                        },
                        headers={
                            'Accept': 'application/json',
                        }
                       )

`requests` returns a `Response` object.  

Often, we want to check that our request was successful.  In a notebook or someother interactive environment, we can just type the name of the variable we have saved our `requests` Response to, in this case the `response` variable.

In [17]:
response

<Response [200]>

  A cleaner and more understandable method is to check the `status_code` attribute.  Both methods return a HTTP status code.  You've probably seen a 404 error when you have tried to access a website that doesn't exist.

In [18]:
response.status_code

200

Try changing `CMR_OPS` to `https://cmr.earthdata.nasa.gov/searches` and run `requests.get` again.  __Don't forget to rerun the cell that assigns the `url` variable__


The response from `requests.get` returns the results of the search and metadata about those results in the `headers`.  

More information about the `response` object can be found by typing `help(response)`.

`headers` contains useful information in a case-insensitive dictionary.  This information is printed below.
*TODO: maybe some context for where the 2 elements k, v, come from?*

In [19]:
for k, v in response.headers.items():
    print(f'{k}: {v}')

Content-Type: application/json;charset=utf-8
Transfer-Encoding: chunked
Connection: keep-alive
Date: Thu, 14 Nov 2024 04:39:53 GMT
X-Frame-Options: SAMEORIGIN
Access-Control-Allow-Origin: *
X-XSS-Protection: 1; mode=block
CMR-Request-Id: 248b3960-c8b1-43d0-be47-27964bb6336f
Content-SHA1: d44a71ae7adafdaada42e662f08eaba2fcafdb5f
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
CMR-Search-After: [0.0,10800.0,"VNP02IMG","2",2105091501,3]
CMR-Hits: 5449
Access-Control-Expose-Headers: CMR-Hits, CMR-Request-Id, X-Request-Id, CMR-Scroll-Id, CMR-Search-After, CMR-Timed-Out, CMR-Shapefile-Original-Point-Count, CMR-Shapefile-Simplified-Point-Count
X-Content-Type-Options: nosniff
CMR-Took: 1787
X-Request-Id: NLJ42Nd-0KisFxkQMtVfn6sMG_cjC1EajDIS8CS0c7dIJ_d5v4PAIQ==
Content-MD5: f09e6719754803684d0e226241594ff4
Vary: Accept-Encoding, User-Agent
Content-Encoding: gzip
Server: ServerTokens ProductOnly
X-Cache: Miss from cloudfront
Via: 1.1 543364398a9b5ee1ed0d763a4bb2c14a.cloud

We can see that the content returned is in `json` format in the UTF-8 character set.  We can also see from `CMR-Hits` that 919 collections were found.

Each item in the dictionary can be accessed in the normal way you access a `python` dictionary but because it is case-insensitive, both

In [20]:
response.headers['CMR-Hits']

'5449'

and

In [21]:
response.headers['cmr-hits']

'5449'

work.

This is a large number of data sets.  I'm going to restrict the search to cloud-hosted datasets from ASF (Alaska SAR Facility) because I'm interested in SAR images of sea ice.  To do this, I set the `provider` parameter to `ASF`.

You can modify the code below to explore all of the cloud-hosted datasets or cloud-hosted datasets from other providers.  A partial list of providers is given below.

DAAC      | Short Name                              | Cloud Provider | On-Premises Provider  
----------|-----------------------------------------|----------------|----------------------  
NSIDC     | National Snow and Ice Data Center       | NSIDC_CPRD     | NSIDC_ECS  
GHRC DAAC | Global Hydrometeorology Resource Center | GHRC_DAAC      | GHRC_DAAC  
PO DAAC   | Physical Oceanography Distributed Active Archive Center | POCLOUD | PODAAC  
ASF       | Alaska Satellite Facility | ASF | ASF  
ORNL DAAC | Oak Ridge National Laboratory | ORNL_CLOUD | ORNL_DAAC  
LP DAAC   | Land Processes Distributed Active Archive Center | LPCLOUD | LPDAAC_ECS
GES DISC  | NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC) | GES_DISC | GES_DISC
OB DAAC   | NASA's Ocean Biology Distributed Active Archive Center |   | OB_DAAC
SEDAC     | NASA's Socioeconomic Data and Applications Center |   | SEDAC

When search by provider, use _Cloud Provider_ to search for cloud-hosted datasets and _On-Premises Provider_ to search for datasets archived at the DAACs.

In [22]:
provider = 'ASF'
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )

In [23]:
response.headers['cmr-hits']

'48'

Search results are contained in the content part of the Response object.  However, `response.content` returns information in bytes.

In [24]:
response.content

b'{"feed":{"updated":"2024-11-14T04:49:13.653Z","id":"https://cmr.earthdata.nasa.gov:443/search/collections.json?cloud_hosted=True&has_granules=True&provider=ASF","title":"ECHO dataset metadata","entry":[{"cloud_hosted":true,"boxes":["-90 -180 90 180"],"has_combine":false,"time_start":"2014-04-03T00:00:00.000Z","version_id":"1","updated":"2022-11-17T22:58:01.000Z","dataset_id":"SENTINEL-1A_SLC","entry_id":"SENTINEL-1A_SLC_1","has_spatial_subsetting":false,"has_transforms":false,"has_variables":false,"data_center":"ASF","short_name":"SENTINEL-1A_SLC","organizations":["ASF","ESA/CS1CGS"],"title":"SENTINEL-1A_SLC","coordinate_system":"CARTESIAN","summary":"Sentinel-1A slant-range product","service_features":{"opendap":{"has_formats":false,"has_variables":false,"has_transforms":false,"has_combine":false,"has_spatial_subsetting":false,"has_temporal_subsetting":false},"esi":{"has_formats":false,"has_variables":false,"has_transforms":false,"has_combine":false,"has_spatial_subsetting":false,"h

It is more convenient to work with `json` formatted data.  I'm using pretty print `pprint` to print the data in an easy to read way.  

__Step through `response.json()`, then to `response.json()['feed']['entry'][0]`__. A reminder that python starts indexing at 0, not 1!

In [26]:
pprint(response.json()['feed']['entry'][1])

{'archive_center': 'ASF',
 'boxes': ['-90 -180 90 180'],
 'browse_flag': False,
 'cloud_hosted': True,
 'consortiums': ['GEOSS', 'EOSDIS'],
 'coordinate_system': 'CARTESIAN',
 'data_center': 'ASF',
 'dataset_id': 'SENTINEL-1B_SLC',
 'entry_id': 'SENTINEL-1B_SLC_1',
 'has_combine': False,
 'has_formats': False,
 'has_spatial_subsetting': False,
 'has_temporal_subsetting': False,
 'has_transforms': False,
 'has_variables': False,
 'id': 'C1327985661-ASF',
 'links': [{'href': 'https://search.asf.alaska.edu/',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#'},
           {'href': 'https://earthdata.nasa.gov/data/catalog/alaska-satellite-facility-distributed-active-archive-center-sentinel-1-version-1/',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#'}],
 'online_access_flag': True,
 'orbit_parameters': {},
 'organizations': ['ASF', 'ESA/CS1CGS'],
 'original_format': 'UMM_JSON',
 'platforms': [

The first response is not the result I am looking for *TODO: because xyz...but it does show a few variables that we can use to further refine the search*.  So I want to print the name of the dataset (`dataset_id`) and the concept id (`id`). We can build this variable and print statement like we did above with the `url` variable.
*TODO: is it worth saying something about what "feed" and "entry" are?*

In [27]:
collections = response.json()['feed']['entry']

In [28]:
for collection in collections:
    print(f'{collection["archive_center"]} {collection["dataset_id"]} {collection["id"]}')

ASF SENTINEL-1A_SLC C1214470488-ASF
ASF SENTINEL-1B_SLC C1327985661-ASF
ASF SENTINEL-1A_DUAL_POL_GRD_HIGH_RES C1214470533-ASF
ASF SENTINEL-1B_DUAL_POL_GRD_HIGH_RES C1327985645-ASF
Alaska Satellite Facility Sentinel-1 Interferograms C1595422627-ASF
ASF SENTINEL-1A_METADATA_SLC C1214470496-ASF
ASF SENTINEL-1A_DUAL_POL_METADATA_GRD_HIGH_RES C1214470576-ASF
ASF SENTINEL-1A_METADATA_RAW C1214470532-ASF
ASF SENTINEL-1A_METADATA_OCN C1266376001-ASF
ASF SENTINEL-1A_OCN C1214472977-ASF


But there is a problem.  We know from `CMR-Hits` that there are 49 datasets but only 10 are printed.  This is because CMR restricts the number of results returned by a query.  The default is 10 but it can be set to a maximum of 2000.  Knowing that there were 49 'hits', I'll set `page_size` to 49. Then, we can re-run our for loop for the collections.

In [29]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'provider': provider,
                            'page_size': 49,
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )

In [30]:
collections = response.json()['feed']['entry']
for collection in collections:
    print(f'{collection["archive_center"]} {collection["dataset_id"]} {collection["id"]}')

ASF SENTINEL-1A_SLC C1214470488-ASF
ASF SENTINEL-1B_SLC C1327985661-ASF
ASF SENTINEL-1A_DUAL_POL_GRD_HIGH_RES C1214470533-ASF
ASF SENTINEL-1B_DUAL_POL_GRD_HIGH_RES C1327985645-ASF
Alaska Satellite Facility Sentinel-1 Interferograms C1595422627-ASF
ASF SENTINEL-1A_METADATA_SLC C1214470496-ASF
ASF SENTINEL-1A_DUAL_POL_METADATA_GRD_HIGH_RES C1214470576-ASF
ASF SENTINEL-1A_METADATA_RAW C1214470532-ASF
ASF SENTINEL-1A_METADATA_OCN C1266376001-ASF
ASF SENTINEL-1A_OCN C1214472977-ASF
ASF SENTINEL-1B_DUAL_POL_GRD_MEDIUM_RES C1327985660-ASF
ASF SENTINEL-1B_METADATA_SLC C1327985617-ASF
ASF SENTINEL-1A_DUAL_POL_GRD_MEDIUM_RES C1214471521-ASF
ASF SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES C1214472336-ASF
ASF SENTINEL-1A_RAW C1214470561-ASF
ASF SENTINEL-1A_SINGLE_POL_GRD_MEDIUM_RES C1214472994-ASF
ASF SENTINEL-1A_SINGLE_POL_METADATA_GRD_HIGH_RES C1214470732-ASF
ASF SENTINEL-1A_SINGLE_POL_METADATA_GRD_MEDIUM_RES C1214473170-ASF
ASF SENTINEL-1B_DUAL_POL_METADATA_GRD_HIGH_RES C1327985741-ASF
ASF SEN

## Granule Search
In NASA speak, Granules are files.  In this example, we will search for recent Sentinel-1 Ground Range Detected (GRD) Medium Resolution Synthetic Aperture Radar images over the east coast of Greenland.  The data in these files are most useful for sea ice mapping.

I'll use the data range 2021-10-17 00:00 to 2021-10-18 23:59:59.

I'll use a simple bounding box to search.
- SW: 76.08166,-67.1746
- NW: 88.19689,21.04862

From the collections search, I know the concept ids for Sentinel-1A and Sentinel-1B GRD medium resolution are
- C1214472336-ASF
- C1327985578-ASF

We need to change the resource url to look for granules instead of collections

In [31]:
url = f'{CMR_OPS}/{"granules"}'

We will search by `concept_id`, `temporal`, and `bounding_box`.  Details about these search parameters can be found in the CMR API Documentation.

The formatting of the values for each parameter is quite specific.  
Temporal parameters are in ISO 8061 format `yyyy-MM-ddTHH:mm:ssZ`.  
Bounding box coordinates are lower left longitude, lower left latitude, upper right longitude, upper right latitude.

In [32]:
response = requests.get(url,
                        params={
                            'concept_id': 'C1214472336-ASF',
                            'temporal': '2020-10-17T00:00:00Z,2020-10-18T23:59:59Z',
                            'bounding_box': '76.08166,-67.1746,88.19689,21.04862',
                            'page_size': 200,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response.status_code)

200


In [33]:
print(response.headers['CMR-Hits'])

6


In [34]:
granules = response.json()['feed']['entry']
#for granule in granules:
#    print(f'{granule["archive_center"]} {granule["dataset_id"]} {granule["id"]}')

In [37]:
for granule in granules:
   print(f'{granule["dataset_id"]} {granule["id"]}')

SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954601581-ASF
SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954616816-ASF
SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954616638-ASF
SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954805829-ASF
SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954799806-ASF
SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES G1954798927-ASF


In [35]:
pprint(granules)

[{'browse_flag': True,
  'collection_concept_id': 'C1214472336-ASF',
  'coordinate_system': 'GEODETIC',
  'data_center': 'ASF',
  'dataset_id': 'SENTINEL-1A_DUAL_POL_METADATA_GRD_MEDIUM_RES',
  'day_night_flag': 'UNSPECIFIED',
  'granule_size': '0.05633258819580078',
  'id': 'G1954601581-ASF',
  'links': [{'href': 'https://datapool.asf.alaska.edu/METADATA_GRD_MD/SA/S1A_EW_GRDM_1SDH_20201017T132009_20201017T132039_034836_040F98_404E.iso.xml',
             'hreflang': 'en-US',
             'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#',
             'title': 'This link provides direct download access to the '
                      'granule.'},
            {'href': 's3://asf-ngap2w-p-s1-xml-8cf7476b/S1A_EW_GRDM_1SDH_20201017T132009_20201017T132039_034836_040F98_404E.iso.xml',
             'hreflang': 'en-US',
             'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#',
             'title': 'This link provides direct download access to the '
                      'granule.'},
      